In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from bayes_opt import BayesianOptimization
import category_encoders as ce

In [2]:
train = pd.read_csv(r'Orga/train.csv', index_col=0)
test = pd.read_csv(r'Orga/test.csv', index_col=0)

In [3]:
#metrica
def MAE(actual, pred):
    return (np.mean(np.absolute(actual - pred)))

## Preprocesamiento

In [4]:
train = train.fillna(0)
test = test.fillna(0)

In [5]:
concat = pd.concat([train, test], sort=False)

In [157]:
train = train.loc[:,['metroscubiertos', 'banos', 'metrostotales', 'garages', 'habitaciones', 'gimnasio', 'usosmultiples', 'piscina', 'idzona', 'precio','fecha']]
test = test.loc[:,['metroscubiertos', 'banos', 'metrostotales', 'garages', 'habitaciones', 'gimnasio', 'usosmultiples', 'piscina', 'idzona','fecha']]

In [158]:
#aplico label encoder para normalizar las claves
encoder = LabelEncoder()
train['idzona'] = encoder.fit_transform(train['idzona']).astype(np.int64)
test['idzona'] = encoder.fit_transform(train['idzona']).astype(np.int64)

In [160]:
encoder = ce.HashingEncoder(cols=['idzona'], n_components=200)
x_train = train.drop('precio', axis = 1)
y_train = train['precio']

x_test = test.drop('precio', axis = 1)
y_test = test['precio']

In [161]:
ids_zonas_train = encoder.fit_transform(x_train, y_train)
ids_zonas_test = encoder.fit_transform(x_test, y_test)

In [162]:
def zonas(self):
    lista = []
    for i in range (0, 200):
        lista.append('col_' + str(i))
        
    return lista

In [163]:
ids_zonas = ids_zonas[zonas]

In [164]:
ids_zonas = ids_zonas.reset_index().rename(columns={'index':'id'})
train = train.reset_index()

In [167]:
train = pd.merge(train, ids_zonas, on='id', how='inner')
#fin encoding

In [168]:
train


,metroscubiertos,banos,metrostotales,garages,habitaciones,gimnasio,usosmultiples,piscina,idzona,precio,...,col_190,col_191,col_192,col_193,col_194,col_195,col_196,col_197,col_198,col_199
id,,,,,,,,,,,,,,,,,,,,,
53461,268.0,2.0,180.0,2.0,3.0,0.0,0.0,0.0,3619,3600000.0,...,0,0,0,0,0,0,0,0,0,0
209067,63.0,1.0,67.0,1.0,2.0,0.0,0.0,0.0,7286,650000.0,...,0,0,0,0,0,0,0,0,0,0
185997,95.0,1.0,95.0,1.0,2.0,0.0,0.0,0.0,5992,1150000.0,...,0,0,0,0,0,0,0,0,0,0
126147,75.0,1.0,90.0,1.0,2.0,0.0,0.0,0.0,3055,1100000.0,...,0,0,0,0,0,0,0,0,0,0
139233,140.0,2.0,160.0,1.0,3.0,0.0,0.0,0.0,12292,1150000.0,...,0,0,0,0,0,0,0,0,0,0
5013,293.0,3.0,293.0,2.0,4.0,0.0,0.0,0.0,18557,4200000.0,...,0,0,0,0,0,0,0,0,0,0
44962,58.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,1833,310000.0,...,0,0,0,0,0,0,0,0,0,0
134537,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9113,6200000.0,...,0,0,0,0,0,0,0,0,0,0
51180,100.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,14178,488000.0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#Agregar feature con año
#Buscar feature hashing, probar con 400 columnas o por ahi (category_encoders biblioteca)
train["fecha"] = pd.to_datetime(train["fecha"])
train["año"] = train["fecha"].dt.year.astype(int)

test["fecha"] = pd.to_datetime(test["fecha"])
test["año"] = test["fecha"].dt.year.astype(int)

In [47]:
train_2012 = train[lambda df: df["año"] == 2012].drop(columns=["fecha","año"])
train_2013 = train[lambda df: df["año"] == 2013].drop(columns=["fecha","año"])
train_2014 = train[lambda df: df["año"] == 2014].drop(columns=["fecha","año"])
train_2015 = train[lambda df: df["año"] == 2015].drop(columns=["fecha","año"])
train_2016 = train[lambda df: df["año"] == 2016].drop(columns=["fecha","año"])

test_2012 = test[lambda df: df["año"] == 2012].drop(columns=["fecha","año"])
test_2013 = test[lambda df: df["año"] == 2013].drop(columns=["fecha","año"])
test_2014 = test[lambda df: df["año"] == 2014].drop(columns=["fecha","año"])
test_2015 = test[lambda df: df["año"] == 2015].drop(columns=["fecha","año"])
test_2016 = test[lambda df: df["año"] == 2016].drop(columns=["fecha","año"])

## Feature Engineering

In [7]:
#train['cant_amenities'] = train['usosmultiples'] + train['piscina'] + train['gimnasio']
#test['cant_amenities'] = test['usosmultiples'] + test['piscina'] + test['gimnasio']

In [6]:
concat['cant_amenities'] = concat['usosmultiples'] + concat['piscina'] + concat['gimnasio']

In [7]:
concat['fecha'] = pd.to_datetime(concat['fecha'])
concat['año'] = concat['fecha'].dt.year

In [8]:
def features_descripcion(lista_features, concat):
    for i in range(len(lista_features)):
        concat[lista_features[i]] = concat['descripcion'].str.contains(lista_features[i], regex=False).astype(bool)
    return concat

In [9]:
lista_features = ['luminoso', 'jardin', 'terraza', 'patio', 'balcón']
concat = features_descripcion(lista_features, concat)

In [10]:
concat['centroscomercialescercanos'] = concat['centroscomercialescercanos'].map({1:2, 0:0})

In [11]:
concat['cercanias'] = concat['centroscomercialescercanos'] + concat['escuelascercanas']
concat['cercanias'] = concat['cercanias'].astype(str)

In [12]:
#concat = concat.join(pd.get_dummies(concat['provincia'], prefix = 'prov'))
del concat['provincia']

In [13]:
concat = concat.join(pd.get_dummies(concat['ciudad'], prefix = 'ciudad'))
del concat['ciudad']

In [14]:
concat = concat.join(pd.get_dummies(concat['tipodepropiedad'], prefix = 'tipo'))
del concat['tipodepropiedad']

In [15]:
concat = concat.join(pd.get_dummies(concat['cercanias'], prefix = 'cercania'))
del concat['cercanias']

In [16]:
del concat['lat']
del concat['lng']
del concat['descripcion']
del concat['titulo']
del concat['direccion']
del concat['idzona']
del concat['fecha']

In [17]:
train = concat.iloc[0:240000]
test = concat.iloc[240000:300000]

In [18]:
del test['precio']

In [19]:
train_2012 = train[train['año'] == 2012]
train_2013 = train[train['año'] == 2013]
train_2014 = train[train['año'] == 2014]
train_2015 = train[train['año'] == 2015]
train_2016 = train[train['año'] == 2016]

test_2012 = test[test['año'] == 2012]
test_2013 = test[test['año'] == 2013]
test_2014 = test[test['año'] == 2014]
test_2015 = test[test['año'] == 2015]
test_2016 = test[test['año'] == 2016]

## Bayesian Opt

In [20]:
def train_set(train,test):
    X = train.drop(['precio'], axis=1)
    y = train['precio']
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)
    print("Train shapes: X = " + str(X_train.shape) + " y = " + str(y_train.shape))
    print("Validation shapes: X = " + str(X_val.shape) +  " y = " + str(y_val.shape))
    print("Test shape: " + str(test.shape))
    return X, y, X_val, y_val

In [21]:
x_2012, y_2012, x_2012_val, y_2012_val = train_set(train_2012,test_2012)
x_2013, y_2013, x_2013_val, y_2013_val = train_set(train_2013,test_2013)
x_2014, y_2014, x_2014_val, y_2014_val = train_set(train_2014,test_2014)
x_2015, y_2015, x_2015_val, y_2015_val = train_set(train_2015,test_2015)
x_2016, y_2016, x_2016_val, y_2016_val = train_set(train_2016,test_2016)

Train shapes: X = (17650, 969) y = (17650,)
Validation shapes: X = (5884, 969) y = (5884,)
Test shape: (5899, 969)
Train shapes: X = (22789, 969) y = (22789,)
Validation shapes: X = (7597, 969) y = (7597,)
Test shape: (7592, 969)
Train shapes: X = (30429, 969) y = (30429,)
Validation shapes: X = (10143, 969) y = (10143,)
Test shape: (10018, 969)
Train shapes: X = (38602, 969) y = (38602,)
Validation shapes: X = (12868, 969) y = (12868,)
Test shape: (13017, 969)
Train shapes: X = (70528, 969) y = (70528,)
Validation shapes: X = (23510, 969) y = (23510,)
Test shape: (23474, 969)


In [22]:
def xgb_evaluate(max_depth, gamma, colsample_bytree,seed,min_child_weight,n_estimators,
                  reg_alpha,reg_lambda,subsample,learning_rate):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': subsample,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
              "seed": int(seed),
              "min_child_weight": min_child_weight,
              "n_estimators": n_estimators,
              "reg_alpha": reg_alpha,
              "reg_lambda": reg_lambda,
              "learning_rate": learning_rate
             }
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=1000, nfold=5)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [23]:
def optimize(sett):
    dtrain = sett
    xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9),
                                             "seed": (10,50),
                                             "min_child_weight": (0.4,1.5),
                                             "n_estimators":(100,10000),                                                                    
                                             "reg_alpha":(0.2,0.75),
                                             "reg_lambda": (0.2,0.8),
                                             "subsample" : (0.3, 0.8),
                                             "learning_rate": (0.01,0.07),
                                            })
    # Se probo con 3 puntos iniciales y 10 iteraciones, deberiamos probar con mas en ambas
    # La prox iteracion en casa probar 100 y 10000
    xgb_bo.maximize(init_points=3, n_iter=10, acq='ei')
    params = xgb_bo.max['params']
    params['max_depth'] = int(params['max_depth'])
    params["seed"] = int(params["seed"])
    params["n_estimators"] = int(params["n_estimators"])
    return params

In [24]:
import xgboost as xgb

dtrain_2012 = xgb.DMatrix(x_2012, label=y_2012)
dtrain_2013 = xgb.DMatrix(x_2013, label=y_2013)
dtrain_2014 = xgb.DMatrix(x_2014, label=y_2014)
dtrain_2015 = xgb.DMatrix(x_2015, label=y_2015)
dtrain_2016 = xgb.DMatrix(x_2016, label=y_2016)

/home/lautaro/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/lautaro/.local/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [27]:
dtrain = train_2012
params_2012 = optimize(dtrain_2012)
params_2013 = optimize(dtrain_2013)
params_2014 = optimize(dtrain_2014)
params_2015 = optimize(dtrain_2015)
params_2016 = optimize(dtrain_2016)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... |   seed    | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------


AttributeError: 'DataFrame' object has no attribute 'num_row'

## XGBoost

In [29]:
def xgboost(x,y,x_val,y_val, test, params):
    best_xgb_model = XGBRegressor(colsample_bytree=params["colsample_bytree"],
                 gamma=params["gamma"],                 
                 learning_rate=params["learning_rate"],
                 max_depth=params["max_depth"],
                 min_child_weight=params["min_child_weight"],
                 n_estimators=params["n_estimators"],                                                                    
                 reg_alpha=params["reg_alpha"],
                 reg_lambda=params["reg_lambda"],
                 subsample=params["subsample"],
                 seed=params["seed"])
    best_xgb_model.fit(x,y)
    XGB_pred_val = best_xgb_model.predict(x_val)
    XGB_mae = MAE(y_val, XGB_pred_val)
    print("MAE XGB: " + str(XGB_mae))
    XGB_pred_test = best_xgb_model.predict(test)
    res = pd.DataFrame(XGB_pred_test, index=test.index, columns=['precio'])
    res = res.rename(columns={'precio':'target'})
    
    return res, XGB_pred_val

In [32]:
res_2012, XGB_2012_pred_val = xgboost(x_2012, y_2012, x_2012_val, y_2012_val, test_2012, params_2012)
res_2013, XGB_2013_pred_val = xgboost(x_2013, y_2013, x_2013_val, y_2013_val, test_2013, params_2013)
res_2014, XGB_2014_pred_val = xgboost(x_2014, y_2014, x_2014_val, y_2014_val, test_2014, params_2014)
res_2015, XGB_2015_pred_val = xgboost(x_2015, y_2015, x_2015_val, y_2015_val, test_2015, params_2015)
res_2016, XGB_2016_pred_val = xgboost(x_2016, y_2016, x_2016_val, y_2016_val, test_2016, params_2016)


NameError: name 'params_2012' is not defined

In [120]:
#ensamblo todo y comparo
#XGB_mae = MAE(y_val_ensamblado, XGB_pred_val_ensamblado)
#print("MAE XGB: " + str(XGB_mae))

res = res_2012.append(res_2013).append(res_2014).append(res_2015).append(res_2016)
res.to_csv("workshop-submission-XGB_2.csv", header=True)

In [98]:
import matplotlib.pyplot as plt
fscores = pd.DataFrame({'X': list(best_xgb_model.get_fscore().keys()), 'Y': list(best_xgb_model.get_fscore().values())})
fscores.sort_values(by='Y').plot.bar(x='X')

AttributeError: 'XGBRegressor' object has no attribute 'get_fscore'

## KNN

In [27]:
def knn(x,y,x_val,y_val, test):
    #for i in range(1, 30):
    knn = KNeighborsRegressor(n_neighbors=2)
    knn.fit(x,y)
    knn_pred_val = knn.predict(x_val)
    knn_mae = MAE(y_val, knn_pred_val)
    print("MAE KNN: " + str(knn_mae))
    #print("Vecinos: " + str(i))
    knn_pred_test = knn.predict(test)
    res = pd.DataFrame(knn_pred_test, index=test.index, columns=['precio'])
    res = res.rename(columns={'precio':'target'})
    
    return res, knn_pred_val

In [28]:
res_2012, knn_2012_pred_val = knn(x_2012, y_2012, x_2012_val, y_2012_val, test_2012)
res_2013, knn_2013_pred_val = knn(x_2013, y_2013, x_2013_val, y_2013_val, test_2013)
res_2014, knn_2014_pred_val = knn(x_2014, y_2014, x_2014_val, y_2014_val, test_2014)
res_2015, knn_2015_pred_val = knn(x_2015, y_2015, x_2015_val, y_2015_val, test_2015)
res_2016, knn_2016_pred_val = knn(x_2016, y_2016, x_2016_val, y_2016_val, test_2016)

MAE KNN: 430215.4999150238
MAE KNN: 445690.47999210213
MAE KNN: 503914.4383318545
MAE KNN: 545653.0558361828
MAE KNN: 602746.5241599319


In [29]:
res = res_2012.append(res_2013).append(res_2014).append(res_2015).append(res_2016)
res.to_csv("workshop-submission-KNN_2.csv", header=True)